In [1]:
import pandas as pd
import numpy as np
import re


## Loading data

First, load all of the data frames I'm going to need and check out their shape

In [2]:
seeds = pd.read_csv("march-madness-2017/TourneySeeds.csv")
seeds.tail()

,Season,Seed,Team
2077,2016,Z13,1423
2078,2016,Z14,1453
2079,2016,Z15,1167
2080,2016,Z16a,1221
2081,2016,Z16b,1380


In [3]:
geog = pd.read_csv("march-madness-2017/TourneyGeog.csv")
geog.drop(["lat", "lng", "host"], axis=1, inplace=True)
geog.head(5)

,season,daynum,wteam,lteam,slot
0,1985,136,1207,1250,R1W1
1,1985,136,1260,1233,R1W4
2,1985,136,1374,1330,R1W5
3,1985,136,1396,1439,R1W8
4,1985,136,1385,1380,R1X1


In [4]:
games = pd.read_csv("march-madness-2017/TourneyCompactResults.csv")
#games = pd.concat([games, slots], ignore_index=True, axis=1)
print(games.shape)
games.head()



(2050, 8)


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [5]:
slots = pd.read_csv("march-madness-2017/TourneySlots.csv")
#slots = slots[(slots["Season"] < 2016)]
slots = slots.reset_index(drop=True)
print(slots.shape)
slots.head(10)

(2050, 4)


,Season,Slot,Strongseed,Weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12
5,1985,R1W6,W06,W11
6,1985,R1W7,W07,W10
7,1985,R1W8,W08,W09
8,1985,R1X1,X01,X16
9,1985,R1X2,X02,X15


In [6]:
print(games.shape, slots.shape)
games = games[games["Season"] > 1999]
#games = pd.concat([games, slots], axis=1).head()
games.head()

(2050, 8) (2050, 4)


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
945,2000,136,1112,71,1238,47,N,0
946,2000,136,1120,72,1166,69,N,0
947,2000,136,1211,77,1257,66,N,0
948,2000,136,1235,88,1148,78,N,0
949,2000,136,1246,85,1382,80,N,2


In [7]:
# do merging of games with seeds and slots 


games = pd.merge(games, seeds, left_on=["Season", "Wteam"], right_on=["Season", "Team"], copy=False)
games.rename(columns={"Seed":"Wseed"}, inplace=True)
print(games.shape)
games = pd.merge(games, seeds, left_on=["Season", "Lteam"], right_on=["Season", "Team"], copy=False)
games.rename(columns={"Seed":"Lseed"}, inplace=True)
print(games.shape)
games = pd.merge(games, geog, left_on=["Season", "Wteam", "Lteam"], right_on=["season", "wteam", "lteam"], copy=False)
print(games.shape)

games.drop(["wteam", "lteam", "Team_x", "Team_y", "Numot", "daynum", "season", "Wloc"], axis=1, inplace=True)

print(games.shape)

(1105, 10)
(1105, 12)
(1105, 17)
(1105, 9)


In [8]:
games = pd.merge(games, slots, left_on=["Season", "slot"], right_on=["Season", "Slot"], copy=False)
games.drop(["slot", "Wscore", "Lscore"], axis=1, inplace=True)
games.head(10)

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,Strongseed,Weakseed
0,2000,136,1112,1238,Z01,Z16,R1Z1,Z01,Z16
1,2000,136,1120,1166,Y07,Y10,R1Y7,Y07,Y10
2,2000,136,1211,1257,Z10,Z07,R1Z7,Z07,Z10
3,2000,138,1211,1385,Z10,Z02,R2Z2,R1Z2,R1Z7
4,2000,136,1235,1148,Y02,Y15,R1Y2,Y02,Y15
5,2000,138,1235,1120,Y02,Y07,R2Y2,R1Y2,R1Y7
6,2000,143,1235,1417,Y02,Y06,R3Y2,R2Y2,R2Y3
7,2000,136,1246,1382,Y05,Y12,R1Y5,Y05,Y12
8,2000,136,1261,1369,Z04,Z13,R1Z4,Z04,Z13
9,2000,138,1261,1400,Z04,Z05,R2Z4,R1Z4,R1Z5


In [9]:
# pre-process games

# drop early games
games = games[games["Daynum"] >  135]
games.rename(columns={"Strongseed":"seed1", "Weakseed": "seed2"}, inplace=True)

day_to_round = {136:1, 137:1, 138:2,139:2, 143:3, 144:3, 145:4, 146:4, 152:5,154:6}
round_to_points = {1:1, 2:2, 3:4, 4:8, 5:16, 6:32}
games["round"] = games["Daynum"].map(day_to_round)
games["bracket_points"] = games["round"].map(round_to_points)

games.head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
0,2000,136,1112,1238,Z01,Z16,R1Z1,Z01,Z16,1,1
1,2000,136,1120,1166,Y07,Y10,R1Y7,Y07,Y10,1,1
2,2000,136,1211,1257,Z10,Z07,R1Z7,Z07,Z10,1,1
3,2000,138,1211,1385,Z10,Z02,R2Z2,R1Z2,R1Z7,2,2
4,2000,136,1235,1148,Y02,Y15,R1Y2,Y02,Y15,1,1


In [10]:
# make sure we have the right nr of games each year

num_games_by_day = games.groupby(["Season", "Daynum"])["Daynum", "Season"].agg(['count'])
games.groupby("Season").agg("count")["Daynum"]

Season
2000    63
2001    63
2002    63
2003    63
2004    63
2005    63
2006    63
2007    63
2008    63
2009    63
2010    63
2011    63
2012    63
2013    63
2014    63
2015    63
2016    63
Name: Daynum, dtype: int64

In [11]:
# do predictions by slots
def make_group(size):
    players = []
    for p in range(size):
        players.append(("p_" +str(p), p/size))
    return players

print(make_group(10))
    

[('p_0', 0.0), ('p_1', 0.1), ('p_2', 0.2), ('p_3', 0.3), ('p_4', 0.4), ('p_5', 0.5), ('p_6', 0.6), ('p_7', 0.7), ('p_8', 0.8), ('p_9', 0.9)]


In [12]:
games_2003 = games[games["Season"] == 2003]
print(games_2003.shape)
games_2003.head(20)

(63, 11)


,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
192,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1,1
193,2003,138,1112,1211,Z01,Z09,R2Z1,R1Z1,R1Z8,2,2
194,2003,143,1112,1323,Z01,Z05,R3Z1,R2Z1,R2Z4,3,4
195,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1,1
196,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1,1
197,2003,136,1143,1301,W08,W09,R1W8,W08,W09,1,1
198,2003,136,1163,1140,X05,X12,R1X5,X05,X12,1,1
199,2003,138,1163,1390,X05,X04,R2X4,R1X4,R1X5,2,2
200,2003,136,1181,1161,Z03,Z14,R1Z3,Z03,Z14,1,1
201,2003,138,1181,1141,Z03,Z11,R2Z3,R1Z3,R1Z6,2,2


In [13]:
def get_empty_bracket(tourney):
    by_round = {}
    for i in range(1,7):
        by_round[i] =  tourney[tourney["round"] == i]
        #print(by_round[i].shape)
    
    return by_round
empty = get_empty_bracket(games_2003)
empty[1].head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
192,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1,1
195,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1,1
196,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1,1
197,2003,136,1143,1301,W08,W09,R1W8,W08,W09,1,1
198,2003,136,1163,1140,X05,X12,R1X5,X05,X12,1,1


In [14]:
def doSub(s):
    #print(s)
    return int(re.sub("[^0-9]", "", s))
    

def do_picks(bracket, player):
    for rnd in bracket.keys():
        
        thisRound = bracket[rnd]

        #print("Round", rnd)
        #print(thisRound)
        if rnd == 1:
            #no need to get games from previous round'
            
            thisRound["seed1_num"] = thisRound["seed1"].apply(doSub)
            thisRound["seed2_num"] = thisRound["seed2"].apply(doSub)
            
            thisRound["pick"] = thisRound.apply(pickGame, axis=1, args=(player[1],))
            bracket[1] = thisRound
        else:
            # first get round and previous round
           
            prev = bracket[rnd-1]
            #print("prev")
            #print(prev)
            
            #merge slot with previous round winner
            thisRound = pd.merge(thisRound, prev.loc[:, ["Slot", "pick"]], left_on=["seed1"], right_on=["Slot"])
            
            thisRound.drop(["seed1", "Slot_y"], axis=1, inplace=True)
            thisRound.rename(columns={"pick":"seed1", "Slot_x": "Slot"}, inplace=True)
            
            #do again for other game
            thisRound = pd.merge(thisRound, prev.loc[:, ["Slot", "pick"]], left_on=["seed2"], right_on=["Slot"])
            thisRound.drop(["seed2", "Slot_y"], axis=1, inplace=True)
            thisRound.rename(columns={"pick":"seed2", "Slot_x":"Slot"}, inplace=True)
            
            
            thisRound["seed1_num"] = thisRound["seed1"].apply(doSub)
            thisRound["seed2_num"] = thisRound["seed2"].apply(doSub)
            #print(thisRound)
            # do picks
            thisRound["pick"] = thisRound.apply(pickGame, axis=1, args=(player[1],))
        
        # add picks to dict
        bracket[rnd] = thisRound
    return bracket
            
def pickGame(g, risk_factor):
    
    # adjust for seed difference, dont pick 16 over 1 as frequently as 10 over 7
    seed_diff = abs(g["seed1_num"] - g["seed2_num"])
    if seed_diff > 2:
        risk_factor /= seed_diff/2
    
    pick_upset = np.random.binomial(1, risk_factor)
    
    first_favorite = g["seed1_num"] < g["seed2_num"]
    #print(risk_factor, pick_upset,  first_favorite, g["seed1_num"] , g["seed2_num"] )
    if pick_upset and first_favorite or not pick_upset and not first_favorite:
        return g["seed2"]
    
    return g["seed1"]
    
    
    

In [15]:
this_bracket = get_empty_bracket(games_2003)
this_bracket = do_picks(this_bracket,  ("b",.5))
pd.concat(this_bracket.values()).tail(10)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Daynum,Lseed,Lteam,Season,Slot,Wseed,Wteam,bracket_points,pick,round,seed1,seed1_num,seed2,seed2_num
5,144,X06,1268,2003,R3X2,X07,1277,4,X02,3,X02,2,X03,3
6,144,W10,1120,2003,R3W2,W03,1393,4,W02,3,W02,2,W11,11
7,144,X05,1163,2003,R3X1,X01,1400,4,X01,3,X01,1,X05,5
0,145,Z01,1112,2003,R4Z1,Z02,1242,8,Z03,4,Z04,4,Z03,3
1,145,Y01,1246,2003,R4Y1,Y03,1266,8,Y01,4,Y01,1,Y03,3
2,146,W01,1328,2003,R4W1,W03,1393,8,W02,4,W05,5,W02,2
3,146,X07,1277,2003,R4X1,X01,1400,8,X02,4,X01,1,X02,2
0,152,Y03,1266,2003,R5YZ,Z02,1242,16,Y01,5,Y01,1,Z03,3
1,152,X01,1400,2003,R5WX,W03,1393,16,X02,5,W02,2,X02,2
0,154,Z02,1242,2003,R6CH,W03,1393,32,Y01,6,X02,2,Y01,1


In [16]:
def is_upset(g):
    #whos the pick
    if g["pick"] == g["seed1"] and g["seed1_num"] > g["seed2_num"] or g["pick"] == g["seed2"] and g["seed1_num"] < g["seed2_num"]:
        return True
    return False
    

In [17]:
tournament = pd.concat(this_bracket.values())

tournament["is_upset"] =  tournament.apply(is_upset, axis=1)


round_to_points = {1:1, 2:2, 3:4, 4:8, 5:16, 6:32}

tournament["bracket_points"] = tournament["round"].map(round_to_points)
tournament["correct_pick"] = tournament["pick"] == tournament["Wseed"]
tournament["points_awarded"] = tournament["bracket_points"] * tournament["correct_pick"]

print(tournament.shape)
tournament.tail(15)

(63, 17)


,Daynum,Lseed,Lteam,Season,Slot,Wseed,Wteam,bracket_points,pick,round,seed1,seed1_num,seed2,seed2_num,is_upset,correct_pick,points_awarded
0,143,Z05,1323,2003,R3Z1,Z01,1112,4,Z04,3,Z01,1,Z04,4,True,False,0
1,143,Z03,1181,2003,R3Z2,Z02,1242,4,Z03,3,Z02,2,Z03,3,True,False,0
2,143,Y02,1338,2003,R3Y2,Y03,1266,4,Y03,3,Y07,7,Y03,3,False,True,4
3,144,W12,1139,2003,R3W1,W01,1328,4,W05,3,W01,1,W05,5,True,False,0
4,143,Y05,1458,2003,R3Y1,Y01,1246,4,Y01,3,Y01,1,Y05,5,False,True,4
5,144,X06,1268,2003,R3X2,X07,1277,4,X02,3,X02,2,X03,3,False,False,0
6,144,W10,1120,2003,R3W2,W03,1393,4,W02,3,W02,2,W11,11,False,False,0
7,144,X05,1163,2003,R3X1,X01,1400,4,X01,3,X01,1,X05,5,False,True,4
0,145,Z01,1112,2003,R4Z1,Z02,1242,8,Z03,4,Z04,4,Z03,3,False,False,0
1,145,Y01,1246,2003,R4Y1,Y03,1266,8,Y01,4,Y01,1,Y03,3,False,False,0


In [18]:
tournament["bracket_points"].sum()

192

In [19]:
def do_preds_for_group(group, games):
    res = []
    for player in group:
        this_bracket = get_empty_bracket(games)
        this_bracket = do_picks(this_bracket,  player)
        tournament = pd.concat(this_bracket.values())

        tournament["is_upset"] =  tournament.apply(is_upset, axis=1)


       
        tournament["correct_pick"] = tournament["pick"] == tournament["Wseed"]
        tournament["points_awarded"] = tournament["bracket_points"] * tournament["correct_pick"]
        
        res.append({"player": player[0], "upsets": tournament["is_upset"].sum(), "points" : tournament["points_awarded"].sum()})
        
        # delete this guys stuff
        del tournament
        del this_bracket
    return pd.DataFrame(res)



In [20]:
gps_100 = []

In [21]:
gps_100.append(do_preds_for_group(make_group(100), games_2003))
gps_100[len(gps_100)-1].sort_values("points", ascending=False).head(10)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,player,points,upsets
93,p_93,137,25
34,p_34,122,17
89,p_89,113,25
78,p_78,90,24
80,p_80,89,24
44,p_44,88,13
31,p_31,88,14
27,p_27,84,8
60,p_60,83,17
28,p_28,83,7


In [22]:
games_2003.head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
192,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1,1
193,2003,138,1112,1211,Z01,Z09,R2Z1,R1Z1,R1Z8,2,2
194,2003,143,1112,1323,Z01,Z05,R3Z1,R2Z1,R2Z4,3,4
195,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1,1
196,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1,1


In [ ]:
def optimal_preds_for_group(games, size, iterations):
    winners = []
    for i in range(iterations):
        brackets = do_preds_for_group(make_group(size), games)
        winners.append(brackets.sort_values("points", ascending=False).head(1))
    return pd.concat(winners)



In [ ]:
groups_of_10_2003 = optimal_preds_for_group(games_2003, 10, 100)
groups_of_10_2003.mean()

In [ ]:
groups_of_10_2003.mean()

In [ ]:
groups_of_100_2003 = optimal_preds_for_group(games_2003, 100, 100)
groups_of_100_2003.mean()

In [ ]:
groups_of_1000_2003 = optimal_preds_for_group(games_2003, 1000, 10)
groups_of_1000_2003.mean()

In [ ]:
def optimal_throughout_time(games):
    seasons = games["Season"].unique()
    
    sums = {}
    for season in seasons:
        these_games = games[games["Season"] == season]
        optimal = optimal_preds_for_group(these_games, 10, 100)
        
        sums[season] = optimal.mean()
        
    return sums

#quick = optimal_throughout_time(games)

In [ ]:
#quick

In [ ]:
5